In [1]:
import os
import sys
import itertools
import datetime
# import pyspark
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

sys.path.insert(0, '../retrosheet')
sys.path.insert(0, '../pybaseball')

from retrosheet import Retrosheet # https://github.com/calestini/retrosheet
import pybaseball as pbb # https://github.com/jldbc/pybaseball
pbb.cache.enable()

In [2]:
class ETL:
    def __init__(self):
        self.SEASON_RANGE = (2017,2017)
        self.GAME_LOOKBACK = [None,3,6,9,12,20,50]
        self.ATBAT_LOOKBACK = [None,20,50,100,300,600]
        
    def extract_retro(self,refresh=False):
        retro_fpath = os.path.join(os.getcwd(),'retrosheet')
        
        if refresh:
            if os.path.exists(retro_fpath)==False:
                os.makedirs(retro_fpath)

            try:
                os.chdir(retro_fpath)
                rs = Retrosheet()
                rs.batch_parse(yearFrom=self.SEASON_RANGE[0], yearTo=self.SEASON_RANGE[1], batchsize=10)
            except:
                raise
            finally:
                os.chdir(retro_fpath+'/..')
        self.retro_data = {
            'RS_BATTING': pd.read_csv(os.path.join(retro_fpath,'batting.csv')),
            'RS_FIELDING': pd.read_csv(os.path.join(retro_fpath,'fielding.csv')),
            'RS_INFO': pd.read_csv(os.path.join(retro_fpath,'info.csv')),
            'RS_LINEUP': pd.read_csv(os.path.join(retro_fpath,'lineup.csv')),
            'RS_PITCHING': pd.read_csv(os.path.join(retro_fpath,'pitching.csv')),
            'RS_PLAYS': pd.read_csv(os.path.join(retro_fpath,'plays.csv')),
            'RS_ROSTERS': pd.read_csv(os.path.join(retro_fpath,'rosters.csv')),
            'RS_RUNNING': pd.read_csv(os.path.join(retro_fpath,'running.csv')),
            'RS_TEAMS': pd.read_csv(os.path.join(retro_fpath,'teams.csv')),
        }
        
    def extract_pbb(self):
        team_lkup = pbb.teamid_lookup.team_ids()[['yearID','teamID','teamIDBR','teamIDretro']]
        team_lkup = team_lkup.loc[
            (team_lkup.yearID>=self.season_range[0])
            & (team_lkup.yearID<=self.season_range[1])
        ]
        player_lkup = None
        
        data = []
        for year_team in team_lkup.values:
            try:
                data.append(
                    pbb.schedule_and_record(year_team[0],year_team[2])
                    .assign(SEASON=year_team[0])
                )
            except:
                # print(year_team)
                # raise
                pass

        self.pbb_data = {
            'SCHEDULE_RECORD': pd.concat(data).merge(
                team_lkup,how='inner',
                left_on=['Tm','SEASON'],
                right_on=['teamIDBR','yearID'],
            ),
        }
    
    def game(self):
        game = self.retro_data['RS_INFO'].pivot(index=['game_id'],columns=['var'],values=['value'])
        game.columns = game.columns.droplevel()
        game = game.reset_index().merge(
            self.retro_data['RS_LINEUP'].add_prefix('home_').loc[self.retro_data['RS_LINEUP'].home_away=='home'],
            how='inner',left_on='game_id',right_on='home_game_id',
        ).merge(
            self.retro_data['RS_LINEUP'].add_prefix('away_').loc[self.retro_data['RS_LINEUP'].home_away=='away'],
            how='inner',left_on='game_id',right_on='away_game_id',
        ).rename({
            'hometeam':'HOME_TEAM',
            'hometeam_score':'HOME_SCORE',
            'visteam':'AWAY_TEAM',
            'awayteam_score':'AWAY_SCORE',
        },axis=1)
        game.columns = [i.upper() for i in game.columns]
        game = game[[
            'GAME_ID','DATE',
            'HOME_TEAM','HOME_SCORE',
            'HOME_1','HOME_2','HOME_3',
            'HOME_4','HOME_5','HOME_6',
            'HOME_7','HOME_8','HOME_9',
            'AWAY_TEAM','AWAY_SCORE',
            'AWAY_1','AWAY_2','AWAY_3',
            'AWAY_4','AWAY_5','AWAY_6',
            'AWAY_7','AWAY_8','AWAY_9',
        ]]
        game.DATE = pd.to_datetime(game.DATE)
        game['SEASON'] = game['DATE'].dt.year
        for i in ['HOME','AWAY']:
            col = '{}_SCORE'.format(i)
            game[col] = game[col].astype(int)
        game['WINNER'] = game.apply(
            lambda x: 'TIE'
            if x['HOME_SCORE']==x['AWAY_SCORE']
            else (
                x['HOME_TEAM']
                if x['HOME_SCORE']>x['AWAY_SCORE']
                else x['AWAY_TEAM']
            )
            ,axis=1)
        game['HOME_DIFF'] = game['HOME_SCORE'] - game['AWAY_SCORE']
        game['AWAY_DIFF'] = game['AWAY_SCORE'] - game['HOME_SCORE']
        game['GAME_IND'] = 1
        self.GAME = game
    
    def team(self):
        idx_cols = ['TEAM_ID','SEASON','DATE','GAME_ID','GAME_NUM','PREGAME_NUM','WINNER']
        _cols_map = {
            '{ME}_TEAM':'TEAM_ID',
            'SEASON':'SEASON',
            'GAME_ID':'GAME_ID',
            'DATE':'DATE',
            '{ME}_DIFF':'SCORE_DIFF',
            '{ME}_SCORE':'RUNS',
            '{YOU}_SCORE':'RUNS_AGAINST',
            'WINNER':'WINNER',
            'GAME_IND':'GAME_IND',
        }
        _ = []
        for home_away in [['HOME','AWAY'],['AWAY','HOME']]:
            _cols = {i.format(ME=home_away[0],YOU=home_away[1]):j for i,j in _cols_map.items()}
            _.append(self.GAME[_cols.keys()].rename(_cols,axis=1))
        team_game = pd.concat(_)
        team_game['WIN_IND'] = team_game.apply(lambda x: 1 if x['TEAM_ID']==x['WINNER'] else 0,axis=1)
        
        team_game = team_game.sort_values(by=['TEAM_ID','SEASON','DATE','GAME_ID'],ascending=True).reset_index(drop=True)
        team_game['PREGAME_NUM'] = team_game.groupby(idx_cols[:2]).cumcount()
        team_game['GAME_NUM'] = team_game['PREGAME_NUM']+1
        
        metrics = {
            'SCORE_DIFF':'SCORE_DIFF',
            'RUNS':'RUNS',
            'RUNS_AGAINST':'RUNS_AGAINST',
            'WIN_IND':'WINS',
            'GAME_IND':'GAMES',
        }
        
        _grouped = team_game.groupby(idx_cols[:2])[ list(metrics.keys()) ]
        for last_n_games in self.GAME_LOOKBACK:
            if last_n_games is None:
                suffix = 'CUM'
                last_n_games = 100**100
            else:
                suffix = 'LAST{}'.format(last_n_games)
            roll = _grouped.rolling(last_n_games,min_periods=0)
            
            for calc in ['AVG','SUM']:
                cols = [ '_'.join([i,suffix,calc]) for i in metrics.values() ]
                if calc=='AVG':
                    team_game[cols] = roll.mean().reset_index(drop=True)
                elif calc=='SUM':
                    team_game[cols] = roll.sum().reset_index(drop=True)
            
                team_game['WIN_RATE_{}_{}'.format(suffix,calc)] = round(
                    team_game['WINS_{}_{}'.format(suffix,calc)]
                    / team_game['GAMES_{}_{}'.format(suffix,calc)]
                ,3)
        
        window_cols = [
            i for i in team_game.columns
            if i not in idx_cols+list(metrics.keys())
        ]
        
        team_game = (
            team_game[idx_cols+list(metrics.keys())]
            .merge(
                team_game[
                    ['TEAM_ID','SEASON','GAME_NUM']+ window_cols
                ].rename({'GAME_NUM':'PREGAME_NUM'},axis=1),
                how='left',on=['TEAM_ID','SEASON','PREGAME_NUM'],
            )
        )
        
#         ranked = self.rank_teams(team_game)
#         rank_cols = [i for i in ranked.columns if i not in ['TEAM_ID','DATE']]
#         rank_diff_cols = [i.replace('RANK','RANK_DIFF') for i in rank_cols]
#         team_game = team_game.merge(ranked,how='inner',on=['TEAM_ID','DATE'])
        
#         _opp_rank = team_game[idx_cols].merge(team_game,how='inner',on='GAME_ID',suffixes=(None,'_OPP'))
#         _opp_rank = _opp_rank.loc[_opp_rank.TEAM_ID!=_opp_rank.TEAM_ID_OPP,idx_cols+rank_cols]

#         _rank_diffs = []
#         for rank,rank_diff in zip(rank_cols,rank_diff_cols):
#             _rank_diffs.append(team_game[rank] - _opp_rank[rank])
#         team_game = pd.concat([team_game]+_rank_diffs,axis=1)

        self.TEAM = team_game.set_index(idx_cols)
    
    def rank_teams(self,team_game):
        rank_idx = ['TEAM_ID','DATE']
        suffixes = ['LAST{}'.format(i) if i else 'CUM' for i in self.GAME_LOOKBACK]
        calcs = ['SUM','AVG']
        rankers = ['WIN_RATE','WINS','SCORE_DIFF','RUNS',]
        rank_types = {'SORT':None,'AVG':'average','MIN':'min',}#'FRST':'first','DNS':'dense',}
        fields = {}
        for suffix in suffixes:
            for calc in calcs:
                for rank_type,method in rank_types.items():
                    if rank_type=='SORT':
                        field = 'RANK_{}_{}_{}'.format(rank_type,suffix,calc)
                        fields[field] = {
                            'type': rank_type,
                            'rankers': ['{}_{}_{}'.format(ranker,suffix,calc) for ranker in rankers],
                        }
                    else:
                        for ranker in rankers:
                            field = 'RANK_{}_{}_{}_{}'.format(rank_type,ranker,suffix,calc)
                            fields[field] = {
                                'type': rank_type,
                                'method': method,
                                'col': '_'.join([ranker,suffix,calc]),
                            }
        
        _ranked_days = []
        for date in team_game.DATE.unique():
            _teams = team_game.loc[team_game.DATE<=date].sort_values(['DATE','GAME_NUM'])
            _teams = _teams.drop_duplicates('TEAM_ID',keep='last')
            _teams['DATE'] = date
            _teams_idx = _teams.set_index(rank_idx)
            
            _ranked_cols = []
            for field_name,info in fields.items():
                if info['type']=='SORT':
                    _ranked_cols.append(
                        _teams
                        .sort_values(info['rankers'],ascending=False)
                        .reset_index(drop=True)
                        .reset_index(drop=False)
                        .set_index(rank_idx)
                        ['index']
                        .rename(field_name)
                        +1
                    )
                else:
                    _ranked_cols.append(
                        _teams_idx[info['col']]
                        .rank(method=info['method'],ascending=False)
                        .rename(field_name)
                    )
            
            _ranked_days.append(pd.concat(_ranked_cols,axis=1))
            
        rank_df = pd.concat(_ranked_days).reset_index(drop=False)
        return rank_df
    
    def load(self):
        output_fpath = os.path.join(os.getcwd(),'output')
        
        self.team.to_csv(os.path.join(output_fpath,'team.csv'))
        self.game.to_csv(os.path.join(output_fpath,'game.csv'))

In [3]:
etl = ETL()

In [4]:
etl.extract_retro(refresh=False)

In [5]:
etl.game()

In [6]:
etl.team()